In [57]:
import pandas as pd
import plotly as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages

In [58]:
def read_fixture_sec(fixture: str, rule='S'):
    # Legge il CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    
    # Resample il DataFrame usando la frequenza specificata (secondi 'S')
    df_secondly = df.resample(rule).sum()
    
    # Creare un DataFrame vuoto con il range di date specificato a livello di secondi
    date_rng = pd.date_range(start='2019-01-01', end='2019-12-31 23:59:59', freq='S')
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame secondario con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_secondly, how='left').fillna(0)
    
    return df_final

In [59]:
def range_plot_media(df,date,perc_val):
    #Media delle emissioni degli intervalli di X secondi
    df=df.resample('30S').mean()
    max_value = df['consumption'].max()

    #if max_value==0:return
    #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
    und_value=(max_value/100)*perc_val
    for index, row in df.iterrows():
        if row['consumption'] <= und_value:
            df.at[index, 'consumption'] = 0

    plt.figure(figsize=(10, 6))

    #Grafico per tutte le emissioni
    plt.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

    #Formatta le date sull'asse x
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

    #Tick per i giorni con colore differenti
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.tick_params(axis='x', which='minor', colors='red')

    # Impostazioni dell'asse y con tick ogni 100 unità
    plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
    max_value = df['consumption'].max()
    if max_value>1000:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
    else:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))

    #Impostazioni della griglia
    plt.xlabel('Date')
    plt.ylabel('Consumption (L)')
    plt.title(f'Emissioni del giorno {date}')
    plt.legend()
    plt.grid(True, which='major', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', linestyle='-', linewidth=5)
    plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
    plt.xticks(rotation=90)
    plt.show() #X CONTROLLI GRAFIIC

    # Chiudere il grafico per liberare memoria
    plt.close()

In [60]:
def pattern_comportamento(df,perc_val):
    #Intervalli sul grafico 770sec, media valori ogni 30sec
    df_max =df.resample('30S').mean()
    max_value = df_max['consumption'].max()

    #if max_value==0:return "0"#0 emissioni tutto il giorno

    #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
    und_value=(max_value/100)*perc_val
    for index, row in df.iterrows():
        if row['consumption'] <= und_value:
            df.at[index, 'consumption'] = 0

    df_inter =df.resample('770S').mean() #abbiamo solo emissioni 'importanti'
    
    #VEDE GLI INTERVALLI E DURATA
    count_inter = 0
    in_interval = False
    start_date = None
    end_date = None
    duration=0
    intervals = []
    for date, consumption in df_inter['consumption'].items():
        if consumption > 0:#abbiamo delle emissioni
            if not in_interval:#inizio intervallo
                in_interval = True
                start_date = date
                duration = 1 
                count_inter += 1
            else:#continuo intervallo
                duration += 1
        else:#no emissioni
            if in_interval:
                in_interval = False
                end_date=date
                intervals.append((start_date, end_date, duration))    
    if in_interval:
        end_date=date
        intervals.append((start_date, end_date, duration))
    print('DEBUG ARRAY INTERVALLI\n')
    print(intervals)

    #un intervallo vale 770s=12,8m
    #CREAZIONE PATTERN
    count_intervalli=len(intervals)
    pattern=""
    #if count_intervalli>3: pattern += "5"#MOLTE EMISSIONI
    #else: pattern += str(count_intervalli) #1,2,3 

    pattern += str(count_intervalli)

    #AGGIUNTA SULL'ANANLISI PER MENO DI 4 INTERVALLI
    if count_intervalli<=3:
        for date_inizio,date_fine, durata in intervals:
            if durata>3: pattern += "L"#INTERVALLO LUNGO
            elif durata>1:  pattern += "M"#INTERVALLO MEDIO
            else: pattern += "P"#INTERVALLO PICCOLO     

    pattern += "_"   
    
    #CREAZIONE ARRAY CON DATA INIZIO E FINE PER FUTURI CALCOLI
    diff_tra_intervalli = []
    dates = [(interval[0]).to_pydatetime() for interval in intervals]#array inizio intervalli
    dates_fine = [(interval[1]).to_pydatetime() for interval in intervals]#array fine intervalli
    
    #CALCOLA TIPI DI INTERVALLI TRA I MOMENTI DI USO DEL DISPOSITIVO
    for i in range(len(dates_fine) - 1):
        difference  = (dates[i+1] - dates_fine[i]).total_seconds()
        diff=difference/770
        diff_tra_intervalli.append(diff)
    print(f'\nspazio tra gli intervalli: {diff_tra_intervalli}\n')
    count_l=0
    count_m=0
    count_p=0
    for x in diff_tra_intervalli:#calcolo dei tipi di intervalli
        if x>3: count_l += 1#intervallo di tempo LUNGO
        elif x>1:  count_m += 1#intervallo di tempo MEDIO
        else: count_p += 1#intervallo di tempo PICCOLO

    #ANALISI GENERALE INTERVALLI
    if count_l==len(dates): pattern+="S0"#MOLTO MOLTO SPARSO
    elif count_l>=(3/4)*len(dates):#MOLTO SPARSO
        if count_m>=(3/4)*count_p: pattern+="S1a"
        elif count_m>=(1/2)*count_p: pattern+="S1b"
        elif count_m>=(1/4)*count_p: pattern+="S1c"
        elif count_m==0: pattern+="S1d"
    elif count_l>=(1/2)*len(dates):#SPARSO
        if count_m>=(3/4)*count_p: pattern+="S2a"
        elif count_m>=(1/2)*count_p: pattern+="S2b"
        elif count_m>=(1/4)*count_p: pattern+="S2c"
        elif count_m==0: pattern+="S2d"
    elif count_l>=(1/4)*len(dates):#MEDIAMENTE SPARSO
        if count_m>=(3/4)*count_p: pattern+="M1a"
        elif count_m>=(1/2)*count_p: pattern+="M1b"
        elif count_m>=(1/4)*count_p: pattern+="M1c"
        elif count_m==0: pattern+="M1d"
    elif count_l==0:#NON MOLTO SPARSO
        if count_m>=(3/4)*count_p: pattern+="M2a"
        elif count_m>=(1/2)*count_p: pattern+="M2b"
        elif count_m>=(1/4)*count_p: pattern+="M2c"
        elif count_m==0: pattern+="P1" #CONCENTRATO
    return pattern

    #logica
    #if x>=(3/4)*y:
    #elif x>=(1/2)*y:
    #elif x>=(1/4)*y:
    #elif x==0:

In [61]:
def pattern_comportamento_test(df):
    #Intervalli sul grafico 770sec=12,8m, media valori ogni 30sec

    df_inter =df.resample('770S').mean() #abbiamo solo emissioni 'importanti'
    
    #VEDE GLI INTERVALLI E DURATA
    in_interval = False
    start_date = None
    end_date = None
    duration=0
    intervals = []
    for date, consumption in df_inter['consumption'].items():
        if consumption > 0:#abbiamo delle emissioni
            if not in_interval:#inizio intervallo
                in_interval = True
                start_date = date
                duration = 1 
            else:#continuo intervallo
                duration += 1
        else:#no emissioni
            if in_interval:
                in_interval = False
                end_date=date
                intervals.append((start_date, end_date, duration))    
    if in_interval:
        end_date=date
        intervals.append((start_date, end_date, duration))
    #DEBUG print(intervals)

    #CREAZIONE PATTERN
    count_intervalli=len(intervals)
    pattern= str(count_intervalli) #NUMERO DI INTERVALLI 

    #AGGIUNTA SULL'ANANLISI PER MENO DI 4 INTERVALLI
    if count_intervalli<4:#osservazione potrei farlo per tutti o allargare il numero 
        for _,_, durata in intervals:
            if durata>3: pattern += "L"#INTERVALLO LUNGO
            elif durata>1:  pattern += "M"#INTERVALLO MEDIO
            else: pattern += "P"#INTERVALLO PICCOLO     

    pattern += "_"   
    
    #CHANGE
    count_l=0
    count_m=0
    count_p=0
    for i in range(len(intervals) - 1):
        ris=int(( (intervals[i+1][0]).to_pydatetime() - (intervals[i][1]).to_pydatetime() ).total_seconds()/770)
        if ris>3: count_l += 1#intervallo di tempo LUNGO
        elif ris>1:  count_m += 1#intervallo di tempo MEDIO
        else: count_p += 1#intervallo di tempo PICCOLO
        #diff_tra_intervalli.append(  ris )

    tot=count_l+(count_m+count_p)
    #ANALISI GENERALE INTERVALLI
    if count_l==tot: pattern+="S0"#MOLTO MOLTO SPARSO
    elif count_l>=(3/4)*tot:#MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="S1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="S1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="S1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="S1d"
        elif count_m>0: pattern+="S1e"
        elif count_m==0: pattern+="S1f"
    elif count_l>=(1/2)*tot:#SPARSO
        if count_m==(count_m+count_p): pattern+="S2a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="S2b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="S2c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="S2d"
        elif count_m>0: pattern+="S2e"
        elif count_m==0: pattern+="S2f"
    elif count_l>=(1/4)*tot:#MEDIAMENTE SPARSO
        if count_m==(count_m+count_p): pattern+="M1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="M1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="M1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="M1d"
        elif count_m>0: pattern+="M1e"
        elif count_m==0: pattern+="M1f"
    elif count_l>0:#NON MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="M2a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="M2b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="M2c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="M2d"
        elif count_m>0: pattern+="M2e"
        elif count_m==0: pattern+="P0" #CONCENTRATO
    elif count_l==0:#NON MOLTO SPARSO
        if count_m==(count_m+count_p): pattern+="P1a"
        elif count_m>=(3/4)*(count_m+count_p): pattern+="P1b"
        elif count_m>=(1/2)*(count_m+count_p): pattern+="P1c"
        elif count_m>=(1/4)*(count_m+count_p): pattern+="P1d"
        elif count_m>0: pattern+="P1e"
        elif count_m==0: pattern+="P1f" #CONCENTRATO
    return pattern

In [62]:
from collections import Counter
#chek queste date array=[('2019-05-10') ,('2019-08-10'),('2019-10-08'),('2019-01-10'),('2019-10-02'),('2019-08-03'),('2019-12-01  ')]

df_dispositivo_sec=read_fixture_sec('Washbasin')
diz_patterns = {}
dizionario_pattern = {}
perc_val=1

start_date = pd.Timestamp('2019-01-07')#PRIMO LUNEDI 
end_date = pd.Timestamp('2019-12-29')#ULTIMA DOMENICA 
date_rng = pd.date_range(start= start_date, end=end_date, freq='D')

for date in date_rng:
    #print(f'{date}\n')
    start_date = pd.Timestamp(date)
    end_date = pd.Timestamp(date).replace(hour=23, minute=59, second=59)
    patterns=""

    #CAPISCE ALGORTIMO
    df= df_dispositivo_sec[start_date:end_date]
    df_max =df.resample('30S').mean()
    max_value = df_max['consumption'].max()
    und_value=(max_value/100)*perc_val
    #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
    for index, row in df.iterrows():
        if row['consumption'] < und_value:
            df.at[index, 'consumption'] = 0
    
    if max_value!=0:
        #GRAFICI range_plot_media(df,start_date,perc_val)
        patterns=pattern_comportamento_test(df)
    else: patterns="0"
    #DEBUG print(f'Pattern grafico {patterns}\n')

    #CODICE PER OTTENERE LE VOLTE CHE UN PATTERN SI PRESENTA
    #Calcola presenze dei pattern
    if patterns in diz_patterns:#se lo trova 
        dizionario_pattern[patterns].append(df_max)
        diz_patterns[patterns] += 1
    else:
        dizionario_pattern[patterns]=[]
        dizionario_pattern[patterns].append(df_max)
        diz_patterns[patterns] = 1
    #CARTELLA DOVE SALVARE I GRAFICI patterngrafico
print(f'Numero di chiavi nel dizionario: {len(diz_patterns)}:{len(dizionario_pattern)}')
print(diz_patterns)
for key,value in dizionario_pattern.items():
        print(f'\n{key}')
        pdf_filename = f'patterngrafico\{key}.pdf'
        with PdfPages(pdf_filename) as pdf:
            for graf in value:
                # Stampa l'oggetto Figure 
                if not isinstance(graf, pd.DataFrame) or 'consumption' not in graf.columns:#CONTROLLO 
                    print(f'Warning: Invalid DataFrame for key {key}')
                    continue

                #Media delle emissioni degli intervalli di X secondi
                df=graf.resample('30S').mean()

                plt.figure(figsize=(10, 6))

                #Grafico per tutte le emissioni
                plt.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

                #Formatta le date sull'asse x
                plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

                #Tick per i giorni con colore differenti
                plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
                plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
                plt.tick_params(axis='x', which='minor', colors='red')

                # Impostazioni dell'asse y con tick ogni 100 unità
                plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
                max_value = df['consumption'].max()
                if max_value>1000:
                    plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
                else:
                    plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))

                #Impostazioni della griglia
                plt.xlabel('Date')
                plt.ylabel('Consumption (L)')
                plt.title(f'Emissioni del pattern {key}')
                plt.legend()
                plt.grid(True, which='major', linestyle='-', linewidth=1)
                plt.grid(True, which='minor', linestyle='-', linewidth=5)
                plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
                plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
                plt.xticks(rotation=90)

                pdf.savefig() #X CONTROLLI GRAFIIC

                # Chiudere il grafico per liberare memoria
                plt.close()
        #print(f'Fine per {key}')


Numero di chiavi nel dizionario: 108:108
{'0': 124, '1P_S0': 4, '3PMM_S2f': 1, '6_S2a': 5, '3LPP_S0': 2, '2LM_S0': 3, '8_S2c': 3, '5_S1a': 5, '9_S2a': 3, '5_S0': 6, '3MPP_S2f': 2, '9_M1c': 1, '3LMP_S0': 3, '6_M1d': 2, '4_S2a': 11, '5_S1f': 7, '5_S2a': 4, '2MP_S0': 8, '3PPL_S0': 1, '3MMP_S2a': 1, '1M_S0': 8, '3PPL_P1c': 1, '7_S2f': 3, '2PP_P1a': 2, '2MM_S0': 6, '8_S2f': 3, '3PPM_S0': 1, '4_S0': 7, '13_M1d': 1, '3PLM_S0': 1, '8_S1a': 5, '8_S2a': 2, '8_S0': 1, '2PP_S0': 8, '2LP_S0': 1, '3MPP_S0': 2, '2PL_P1a': 1, '3MMM_S0': 1, '7_S2d': 1, '7_S1a': 2, '3MMM_S2f': 1, '7_S1f': 4, '7_S2c': 5, '5_M1f': 1, '3PMP_S2f': 2, '5_M1c': 1, '6_S0': 2, '9_M1b': 1, '2MM_P1f': 1, '6_S1a': 2, '4_M1c': 2, '6_S2f': 1, '2PL_S0': 1, '3MPM_S0': 2, '2MP_P1f': 2, '3PPP_S2a': 1, '1L_S0': 2, '4_S2f': 10, '6_S2c': 2, '9_S2c': 3, '4_M1f': 2, '5_M1d': 3, '7_M1b': 2, '12_S2c': 1, '8_M1c': 1, '10_M1c': 1, '6_M1a': 2, '10_M2c': 1, '3MMM_P1c': 1, '16_M1d': 1, '11_S2c': 1, '12_S2d': 2, '11_M2c': 1, '3MPP_S2a': 1, '9_S1c': 

In [63]:
dizionario1={'0': 124, '1P_S0': 4, '3PMM_S2e': 1, '6_S2a': 7, '3LPP_S0': 2, '2LM_S0': 3, '8_S2a': 5, '5_S1a': 5, '9_S2a': 6, '5_S0': 6, '3MPP_S2e': 2, '9_M1a': 2, '3LMP_S0': 3, '6_M1b': 2, '4_S2a': 11, '5_S1e': 7, '5_S2a': 5, '2MP_S0': 8, '3PPL_S0': 1, '3MMP_S2a': 1, '1M_S0': 8, '3PPL_M2a': 1, '7_S2e': 3, '2PP_M2a': 2, '2MM_S0': 6, '8_S2e': 3, '3PPM_S0': 1, '4_S0': 7, '13_M1c': 1, '3PLM_S0': 1, '8_S1a': 5, '8_S0': 1, '2PP_S0': 8, '2LP_S0': 1, '3MPP_S0': 2, '2PL_M2a': 1, '3MMM_S0': 1, '7_S2b': 1, '7_S1a': 2, '3MMM_S2e': 1, '7_S1e': 4, '7_S2a': 5, '5_M1e': 1, '3PMP_S2e': 2, '5_M1a': 1, '6_S0': 2, '2MM_P1': 1, '6_S1a': 2, '4_M1a': 3, '6_S2e': 1, '2PL_S0': 1, '3MPM_S0': 2, '2MP_P1': 2, '3PPP_S2a': 1, '1L_S0': 2, '4_S2e': 10, '4_M1e': 2, '5_M1b': 3, '7_M1a': 2, '12_S2a': 1, '8_M1a': 2, '10_M1a': 2, '6_M1a': 3, '10_': 1, '3MMM_M2a': 1, '16_M1c': 1, '11_S2a': 1, '12_S2c': 1, '11_': 1, '3MPP_S2a': 1, '9_S1a': 2, '12_M1a': 1, '3PLP_S0': 1, '13_S2b': 1, '14_S2a': 1, '12_S2b': 1, '7_M1c': 1, '7_S0': 2, '9_S1e': 1, '10_S1a': 1, '2PM_P1': 1, '16_M1a': 1, '3PLP_S2a': 1, '10_M1c': 1, '2PM_M2a': 1, '4_M2a': 1, '3LPP_M2a': 1, '2MP_M2a': 2, '10_S2a': 2, '3PPM_M2a': 1, '3PPP_S0': 1, '11_S2b': 1, '11_M1c': 1, '6_S1e': 2, '3MMP_M2a': 1, '3MPM_S2e': 1, '10_S2b': 1, '9_M1b': 1, '3LPP_S2e': 1}
dizionario2={'0': 124, '1P_S0': 4, '3PMM_S2e': 1, '6_S2a': 7, '3LPP_S0': 2, '2LM_S0': 3, '8_S2a': 5, '5_S1a': 5, '9_S2a': 6, '5_S0': 6, '3MPP_S2e': 2, '9_M1a': 2, '3LMP_S0': 3, '6_M1b': 2, '4_S2a': 11, '5_S1e': 7, '5_S2a': 5, '2MP_S0': 8, '3PPL_S0': 1, '3MMP_S2a': 1, '1M_S0': 8, '3PPL_M2a': 1, '7_S2e': 3, '2PP_M2a': 2, '2MM_S0': 6, '8_S2e': 3, '3PPM_S0': 1, '4_S0': 7, '13_M1c': 1, '3PLM_S0': 1, '8_S1a': 5, '8_S0': 1, '2PP_S0': 8, '2LP_S0': 1, '3MPP_S0': 2, '2PL_M2a': 1, '3MMM_S0': 1, '7_S2b': 1, '7_S1a': 2, '3MMM_S2e': 1, '7_S1e': 4, '7_S2a': 5, '5_M1e': 1, '3PMP_S2e': 2, '5_M1a': 1, '6_S0': 2, '2MM_P1': 1, '6_S1a': 2, '4_M1a': 3, '6_S2e': 1, '2PL_S0': 1, '3MPM_S0': 2, '2MP_P1': 2, '3PPP_S2a': 1, '1L_S0': 2, '4_S2e': 10, '4_M1e': 2, '5_M1b': 3, '7_M1a': 2, '12_S2a': 1, '8_M1a': 2, '10_M1a': 2, '6_M1a': 3, '10_': 1, '3MMM_M2a': 1, '16_M1c': 1, '11_S2a': 1, '12_S2c': 1, '11_': 1, '3MPP_S2a': 1, '9_S1a': 2, '12_M1a': 1, '3PLP_S0': 1, '13_S2b': 1, '14_S2a': 1, '12_S2b': 1, '7_M1c': 1, '7_S0': 2, '9_S1e': 1, '10_S1a': 1, '2PM_P1': 1, '16_M1a': 1, '3PLP_S2a': 1, '10_M1c': 1, '2PM_M2a': 1, '4_M2a': 1, '3LPP_M2a': 1, '2MP_M2a': 2, '10_S2a': 2, '3PPM_M2a': 1, '3PPP_S0': 1, '11_S2b': 1, '11_M1c': 1, '6_S1e': 2, '3MMP_M2a': 1, '3MPM_S2e': 1, '10_S2b': 1, '9_M1b': 1, '3LPP_S2e': 1}


if dizionario1 == dizionario2:
    print("True")
else:
    print("False")


True


In [64]:
array=[('2019-06-19') ,('2019-06-26')]
df_dispositivo_sec=read_fixture_sec('Washbasin')
for date in array:
    #print(f'{date}\n')
    start_date = pd.Timestamp(date)
    end_date = pd.Timestamp(date).replace(hour=23, minute=59, second=59)
    patterns=""
    df= df_dispositivo_sec[start_date:end_date]
    df_max =df.resample('30S').mean()
    max_value = df_max['consumption'].max()
    und_value=(max_value/100)*perc_val
    #ELIMINO TUTTE LE EMISSIONI INFERIORI A UNA CERTA SOGLIA
    for index, row in df.iterrows():
        if row['consumption'] < und_value:
            df.at[index, 'consumption'] = 0
    
    patterns=pattern_comportamento_test(df)
    print(f'Pattern grafico {patterns} di {date}\n')

Pattern grafico 10_M2c di 2019-06-19

Pattern grafico 11_M2c di 2019-06-26

